In [ ]:
import pandas as pd
import sys
import numpy as np
from itertools import combinations
pd.set_option('display.max_rows', 100)

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# CSV file with PreRedacted Data

In [ ]:
file_path = r'..\data\TwoSensitive.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

# Instantiate the anonymizer

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

In [ ]:
frequency_value = 'Counts'

In [ ]:
anonymizer = DataAnonymizer(df_data, sensitive_columns=sensitive_list, frequency=frequency_value)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
df_log.to_excel('1CreateLog.xlsx', index=False)

In [ ]:
df_less_than_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_less_than_threshold.to_excel('2LessThanThreshold.xlsx', index=False)

In [ ]:
df_one_count = anonymizer.one_count_redacted()

In [ ]:
df_one_count.to_excel('3OneCountRedact.xlsx', index=False)

In [ ]:
df_one_redact_zero = anonymizer.one_redact_zero()

In [ ]:
df_one_redact_zero.to_excel('4OneRedactZero.xlsx', index=False)

In [ ]:
df_cross_suppression = anonymizer.cross_suppression()

In [ ]:
df_cross_suppression.to_excel('5CrossSuppression.xlsx', index=False)

In [ ]:
df_apply_log = anonymizer.apply_log()

In [ ]:
df_apply_log.to_excel('6ApplyLog.xlsx', index=False)

In [ ]:
df_parent_redact = df_one_redact_zero[(df_one_redact_zero['Grouping'] > 0) & (df_one_redact_zero['RedactBinary'] == 1)]

In [ ]:
redact_parent_name = 'RedactParentBinary'

In [ ]:
df_parent_redact.rename(columns = {'RedactBinary':redact_parent_name}, inplace=True)

In [ ]:
df_parent_redact = df_parent_redact[sensitive_list + [redact_parent_name]]

In [ ]:
sensitive_combinations = sorted([combo for i in range(1, len(sensitive_list) + 1) for combo in combinations(sensitive_list, i)], key=len, reverse=True)

In [ ]:
for sensitive_combination in sensitive_combinations:
    list_combination = list(sensitive_combination)
    df_test = df_parent_redact[df_parent_redact[list(sensitive_combination)].notna().all(axis=1)]
    if (list_combination != sensitive_list) & (not df_test.empty):
        print(sensitive_combination)
        string_combination = ''.join(list_combination)
        df_parent_list = df_parent_redact[list(sensitive_combination) + [redact_parent_name]]
        df_primary = df_one_redact_zero.merge(df_parent_list, on = list_combination, how='left')
        mask = (df_primary['RedactParentBinary'] == 1) & (df_primary["MinimumValue" + string_combination] == df_primary[frequency_value])
        df_one_redact_zero.loc[mask, 'RedactBinary'] = 1
        df_one_redact_zero.loc[mask, 'Redact'] = 'Secondary Suppression'
        df_one_redact_zero.loc[mask, 'RedactBreakdown'] += ', Redacting based on aggregate level redaction'

In [ ]:
display(df_one_redact_zero)

In [ ]:
for sensitive_combination in sensitive_combinations:
    list_combination = list(sensitive_combination)
    if (list_combination != sensitive_list):  
        string_combination = ''.join(list_combination)
        df_redacted = df_one_redact_zero[(df_one_redact_zero['RedactBinary'] == 1) & (df_one_redact_zero['Grouping'] == 0)]
        df_count = df_redacted.groupby(['Grouping'] + list(sensitive_combination))['RedactBinary'].count().reset_index()
        df_one_count = df_count[df_count['RedactBinary'] == 1]
        df_one_count = df_one_count[['Grouping'] + list(sensitive_combination)]
        display(df_one_count)
        df_one_count = df_one_redact_zero.merge(df_one_count, on =['Grouping'] + list(sensitive_combination))
        df_one_count = df_one_count[(df_one_count['RedactBinary'] == 0)]
        df_minimum = df_one_count.groupby(['Grouping'] + list(sensitive_combination))[frequency_value].min().reset_index()
        df_minimum.rename(columns = {frequency_value:'CrossMinimum' + string_combination}, inplace=True)
        df_minimum_value = df_one_redact_zero.merge(df_minimum, on =['Grouping'] + list(sensitive_combination), how='left')
        mask = (df_minimum_value["CrossMinimum" + string_combination] == df_minimum_value[frequency_value])
        df_one_redact_zero.loc[mask, 'RedactBinary'] = 1
        df_one_redact_zero.loc[mask, 'Redact'] = 'Secondary Suppression'
        df_one_redact_zero.loc[mask, 'RedactBreakdown'] += ', Redacting based on aggregate level redaction'

In [ ]:
display(df_one_redact_zero)

In [ ]:
redact_parent_name = 'RedactParentBinary'
df_parent_redact.rename(columns = {'RedactBinary':redact_parent_name}, inplace=True)
df_parent_redact = df_parent_redact[[self.parent_organization] + sensitive_list + [redact_parent_name]]
df_parent_redact.drop_duplicates(inplace=True)
self.df_log = self.df_log.merge(df_parent_redact, on = [self.parent_organization] + self.sensitive_columns, how='left')
self.df_log.loc[(self.df_log[redact_parent_name] == 1), 'RedactBinary'] = 1